In [2]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/samples.ipynb")

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
netMHCIIpan_alleles = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/other/netMHCIIpan_alleles.txt').readlines()]

####Take clean data frames and make in format of netMHCpan; also remove patients that have unpredictable alleles

MHC-II

In [17]:
def classII(subset, population, data_df):
    patient_types_df = data_df
    patients = list(patient_types_df.index)

    dictionary = {}
    for patient in patients:
        patient_dictionary = {}
        success = True
        try:
            # DR - adding two of each for consistency sake with other genes
            alleles = list(patient_types_df.ix[patient][['DRB1_allele1', 'DRB1_allele2']])
            alleles.extend(list(patient_types_df.ix[patient][['DRB1_allele1', 'DRB1_allele2']]))
            if len([a for a in alleles if a in netMHCIIpan_alleles]) == 4:
                patient_dictionary['DR'] = alleles
            else:
                success = False

            # DP
            alleles = []
            DA = list(patient_types_df.ix[patient][['DPA1_allele1', 'DPA1_allele2']])
            DB = list(patient_types_df.ix[patient][['DPB1_allele1', 'DPB1_allele2']])
            if ('-' not in DA) & ('-' not in DB):
                for a in DA:
                    for b in DB:
                        alleles.append('HLA-{0}-{1}'.format(a.strip(), b.strip()))
            if len([a for a in alleles if a in netMHCIIpan_alleles]) == 4:
                patient_dictionary['DP'] = alleles
            else:
                success = False

            # DQ
            alleles = []
            DA = list(patient_types_df.ix[patient][['DQA1_allele1', 'DQA1_allele2']])
            DB = list(patient_types_df.ix[patient][['DQB1_allele1', 'DQB1_allele2']])
            if ('-' not in DA) & ('-' not in DB):
                for a in DA:
                    for b in DB:
                        alleles.append('HLA-{0}-{1}'.format(a.strip(), b.strip()))
            if len([a for a in alleles if a in netMHCIIpan_alleles]) == 4:
                patient_dictionary['DQ'] = alleles
            else:
                success = False
            if success:
                dictionary[patient] = patient_dictionary
        except:
            # weeds out the patients with the failed typing
            print patient
    print len(dictionary.keys())
    pickle.dump(dictionary, open('/cellar/users/ramarty/Data/hla_ii/hla_types/clean/{0}.ClassII.{1}.p'.format(population,
                                                                                                       subset),
                                 'w'))

In [18]:
population = 'TCGA'
# HLAx is excluded because it doesn't have the 'A' alleles
for subset in ['conservative', 'inclusive', 'HLA-HD']:
    print subset
    data_df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/clean/{0}.ClassII.{1}.csv'.format(population,
                                                                                                         subset),
                          index_col=0)
    classII(subset, population, data_df)

conservative
6210
inclusive
7562
HLA-HD
7528


MHC-I

In [20]:
! /cellar/users/ramarty/programs/netMHCpan-3.0/netMHCpan -listMHC > /cellar/users/ramarty/Data/hla_ii/presentation/other/netMHCpan_alleles.txt

In [28]:
netMHCpan_alleles = [x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/presentation/other/netMHCpan_alleles.txt').readlines()[35:]]

In [29]:
netMHCpan_alleles[:5]

['HLA-A01:01', 'HLA-A01:02', 'HLA-A01:03', 'HLA-A01:06', 'HLA-A01:07']

In [30]:
pd.read_csv('/cellar/users/ramarty/Data/hla/residue/matrices/mut/class_i/oncogenes.all.max.txt', index_col=0).head()

,HLA-A01:01,HLA-A01:02,HLA-A01:03,HLA-A01:06,HLA-A01:07,HLA-A01:08,HLA-A01:09,HLA-A01:10,HLA-A01:12,HLA-A01:13,...,HLA-C18:03,HLA-G01:01,HLA-G01:02,HLA-G01:03,HLA-G01:04,HLA-G01:06,HLA-G01:07,HLA-G01:08,HLA-G01:09,HLA-E01:01
BRAF_V600E,12.0,7.5,14.0,12.0,14.0,9.5,12.0,8.5,13.0,1.6,...,10.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,17.0
IDH1_R132H,4.5,3.0,5.0,3.0,3.5,3.5,4.5,2.5,1.6,12.0,...,6.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,14.0
PIK3CA_E545K,7.0,6.0,7.5,6.5,6.0,5.5,7.0,6.0,4.0,1.1,...,17.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,6.5,2.5
PIK3CA_H1047R,5.5,6.0,5.5,6.5,5.0,4.0,5.5,5.0,6.0,12.0,...,1.8,6.5,6.5,6.5,6.5,6.5,6.5,6.5,7.0,9.5
KRAS_G12D,17.0,32.0,15.0,19.0,17.0,16.0,17.0,14.0,13.0,6.5,...,24.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,15.0


In [16]:
pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/clean/TCGA.ClassI.inclusive.csv', index_col=0).head()

,A1,A2,B1,B2,C1,C2
TCGA-02-0003,A*02:01,A*24:02,B*07:02,B*27:05,C*02:02,C*07:02
TCGA-02-0033,A*30:04,A*68:01,B*14:01,B*48:01,C*08:01,C*08:02
TCGA-02-0047,A*02:01,A*32:01,B*15:01,B*15:01,C*03:03,C*03:03
TCGA-02-0055,A*11:01,A*31:01,B*51:01,B*55:01,C*03:03,C*15:02
TCGA-02-2466,A*03:01,A*33:01,B*14:02,B*27:05,C*02:02,C*08:02


In [49]:
def classI(subset, population, data_df):
    patient_types_df = data_df
    patients = list(patient_types_df.index)

    dictionary = {}
    for patient in patients:
        patient_dictionary = {}
        
        alleles = list(patient_types_df.ix[patient][['A1', 'A2', 'B1', 'B2', 'C1', 'C2']])
        
        # reformat
        alleles = ['HLA-{0}{1}'.format(a[0], a[2:]) for a in alleles]
        
        if len([a for a in alleles if a in netMHCpan_alleles]) == 6:
                dictionary[patient] = alleles
                
    print len(dictionary.keys())
    pickle.dump(dictionary, open('/cellar/users/ramarty/Data/hla_ii/hla_types/clean/{0}.ClassI.{1}.p'.format(population,
                                                                                                       subset),
                                 'w'))
    return dictionary

In [50]:
population = 'TCGA'
# HLAx is excluded because it doesn't have the 'A' alleles
for subset in ['conservative', 'inclusive']:
    print subset
    data_df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/hla_types/clean/{0}.ClassI.{1}.csv'.format(population,
                                                                                                         subset),
                          index_col=0)
    dictionary = classI(subset, population, data_df)

conservative
6266
inclusive
7965
